<a href="https://colab.research.google.com/github/cargilgar/Smart-Alarm-using-tinyML/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
import numpy as np
import os
import re
from scipy import stats

# Load Dataset

Load Dataset, count Rows and Columns

In [59]:
data_path = os.path.join(os.getcwd(), "Dataset/")
data_list = sorted(os.listdir(data_path))
data_list[0]


subject_csv = pd.read_csv(os.path.join(data_path, data_list[0]), delimiter=',')
subject_csv

#Rows and Columns
total_rows=len(subject_csv.axes[0]) #===> Axes of 0 is for a row
total_cols=len(subject_csv.axes[1]) #===> Axes of 1 is for a column
print("Number of Rows: "+str(total_rows))
print("Number of Columns: "+str(total_cols))


#subject_csv

Number of Rows: 50
Number of Columns: 6


# Labels
stage (0-5, wake = 0, N1 = 1, N2 = 2, N3 = 3, REM = 5)

In [60]:
#Show labels
# Same labels will be reused throughout the program

subject_csv['labels'].describe()
#subject_csv.hist('Heart Rate')
#subject_csv.describe()

subject_csv['labels'] = subject_csv['labels'].map({0:'Wake',
                             1:'NREM',
                             2:'NREM',
                             3:'NREM',
                             5:'REM',
                             },
                             na_action=None)

#Not labeled values --> NaN

subject_csv

,Time,X,Y,Z,Heart Rate,labels
0,25530.00395,0.133011,-0.976974,0.162323,58,NREM
1,25560.01148,0.132507,-0.975006,0.163269,60,NREM
2,25590.01774,0.130524,-0.974991,0.165192,58,NaN
3,25620.00804,0.133987,-0.975983,0.163788,75,NaN
4,25650.01387,0.131546,-0.976486,0.161316,68,NaN
5,25680.00288,0.133026,-0.976486,0.160858,67,NREM
6,25710.00897,0.131531,-0.974518,0.163254,58,NREM
7,25740.01796,0.132019,-0.974518,0.163254,59,NREM
8,25770.00279,0.132980,-0.975479,0.165222,59,NREM
9,25800.01058,0.130081,-0.977966,0.160812,70,NREM


# Split Dataset
Train, Validation and Test

Split the data
We'll use a (70%, 20%, 10%) split for the training, validation, and test sets. Note the data is not being randomly shuffled before splitting. This is for two reasons.

It ensures that chopping the data into windows of consecutive samples is still possible.
It ensures that the validation/test results are more realistic, being evaluated on data collected after the model was trained.

***ANOTHER SPLITING OPTION WOULD BE TO SEPARATE USERS (Crear nueva columna con nombre usuario?? O manejar cada CSV por separado??)***





In [61]:
column_indices = {name: i for i, name in enumerate(subject_csv.columns)}

n = len(subject_csv)
train_subject_csv = subject_csv[0:int(n*0.7)]
val_subject_csv = subject_csv[int(n*0.7):int(n*0.9)]
test_subject_csv = subject_csv[int(n*0.9):]

num_features = subject_csv.shape[1]

train_subject_csv

,Time,X,Y,Z,Heart Rate,labels
0,25530.00395,0.133011,-0.976974,0.162323,58,NREM
1,25560.01148,0.132507,-0.975006,0.163269,60,NREM
2,25590.01774,0.130524,-0.974991,0.165192,58,NaN
3,25620.00804,0.133987,-0.975983,0.163788,75,NaN
4,25650.01387,0.131546,-0.976486,0.161316,68,NaN
5,25680.00288,0.133026,-0.976486,0.160858,67,NREM
6,25710.00897,0.131531,-0.974518,0.163254,58,NREM
7,25740.01796,0.132019,-0.974518,0.163254,59,NREM
8,25770.00279,0.132980,-0.975479,0.165222,59,NREM
9,25800.01058,0.130081,-0.977966,0.160812,70,NREM


# Normalize Training Data
Next, we need to normalize our features within our training data. Of course there are various ways on how to normalize. Please keep in mind that you use the same normalization algorithm later when feeding new data into your neural network. Otherwise your preditions will be off. On top of the normalization we will also apply rounding to the three features.

In [62]:
# Normalize features for training data set (values between 0 and 1)***
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
train_subject_csv['X'] = train_subject_csv['X'] / train_subject_csv['X'].max()
train_subject_csv['Y'] = train_subject_csv['Y'] / train_subject_csv['Y'].max()
train_subject_csv['Z'] = train_subject_csv['Z'] / train_subject_csv['Z'].max()
train_subject_csv['Heart Rate'] = train_subject_csv['Heart Rate'] / train_subject_csv['Heart Rate'].max()

# Round numbers (4 decimals)
train_subject_csv = train_subject_csv.round({'X': 4, 'Y': 4, 'Z': 4, 'Heart Rate': 4})

train_subject_csv

,Time,X,Y,Z,Heart Rate,labels
0,25530.00395,0.5041,-0.7733,0.4795,0.7733,NREM
1,25560.01148,0.5022,-0.7718,0.4823,0.8000,NREM
2,25590.01774,0.4947,-0.7718,0.4880,0.7733,NaN
3,25620.00804,0.5078,-0.7726,0.4839,1.0000,NaN
4,25650.01387,0.4986,-0.7730,0.4766,0.9067,NaN
5,25680.00288,0.5042,-0.7730,0.4752,0.8933,NREM
6,25710.00897,0.4985,-0.7714,0.4823,0.7733,NREM
7,25740.01796,0.5003,-0.7714,0.4823,0.7867,NREM
8,25770.00279,0.5040,-0.7722,0.4881,0.7867,NREM
9,25800.01058,0.4930,-0.7741,0.4751,0.9333,NREM


# Reshape Data into Segments and Prepare for Keras
The data contained in the dataframe is not ready yet to be fed into a neural network. Therefore we need to reshape it. Let’s create another function for this called “create_segments_and_labels”. This function will take in the dataframe and the label names (the constant that we have defined at the beginning) as well as the length of each record. In our case, let’s go with 80 steps (see constant defined earlier). Taking into consideration the 20 Hz sampling rate, this equals to 4 second time intervals (calculation: 0.05 * 80 = 4). Besides reshaping the data, the function will also separate the features (x-acceleration, y-acceleration, z-acceleration) and the labels (associated activity).

In [87]:
def create_segments_and_labels(subject_csv, time_steps, step, label_name):
    # x, y, z acceleration and Heart Rate as features
    N_FEATURES = 4
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    time_steps = 2
    step = 1
    segments = []
    labels = []
    for i in range(0, int(len(subject_csv) - time_steps), step):
        xs = subject_csv['X'].values[i: i + time_steps]
        ys = subject_csv['Y'].values[i: i + time_steps]
        zs = subject_csv['Z'].values[i: i + time_steps]
        HRs = subject_csv['Heart Rate'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(subject_csv[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys, zs, HRs])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels



In [88]:
# x_train --> Features?
# y_train --> Labels?

TIME_PERIODS = 2
STEP_DISTANCE = 1
LABELS =  ['Wake',
          'NREM',
          'REM',
          'NaN']
        
x_train, y_train = create_segments_and_labels(train_subject_csv,
                                              TIME_PERIODS,
                                              STEP_DISTANCE,
                                              LABELS)

KeyError: ignored

# Create Deep Neural Network Model in Keras
The data is ready in such a format that Keras will be able to process it. I have decided to create a neural network with 3 hidden layers of 100 fully connected nodes each (feel free to play around with the shape of the network or even switch to more complex ones like a convolutional neural network).
Important remark: as you remember, we have reshaped our input data from a 80x3 matrix into a vector of length 240 so that Apple’s Core ML can later process our data. In order to reverse this, our first layer in the neural network will reshape the data into the “old” format. The last two layers will again flatten the data and then run a softmax activation function in order to calculate the probability for each class. Remember, that we are working with six classes in our case (Downstairs, Jogging, Sitting, Standing, Upstairs, Walking).

In [ ]:
model_m = Sequential()
# Remark: since coreml cannot accept vector shapes of complex shape like
# [80,3] this workaround is used in order to reshape the vector internally
# prior feeding it into the network
model_m.add(Reshape((TIME_PERIODS, 3), input_shape=(input_shape,)))
model_m.add(Dense(100, activation='relu'))
model_m.add(Dense(100, activation='relu'))
model_m.add(Dense(100, activation='relu'))
model_m.add(Flatten())
model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())



---



---



---



---



---



# Standarize the data
Our timeseries are already in a single length (176). However, their values are usually in various ranges. This is not ideal for a neural network; in general we should seek to make the input values normalized. For this specific dataset, the data is already z-normalized: each timeseries sample has a mean equal to zero and a standard deviation equal to one. This type of normalization is very common for timeseries classification problems, see Bagnall et al. (2016).

Note that the timeseries data used here are univariate, meaning we only have one channel per timeseries example. We will therefore transform the timeseries into a multivariate one with one channel using a simple reshaping via numpy. This will allow us to construct a model that is easily applicable to multivariate time series.

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

Finally, in order to use sparse_categorical_crossentropy, we will have to count the number of classes beforehand.

In [ ]:
num_classes = len(np.unique(y_train))

Now we shuffle the training set because we will be using the validation_split option later when training.

In [ ]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

Standardize the labels to positive integers. The expected labels will then be 0 and 1.

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

# Visualize the data
Here we visualize one timeseries example for each class in the dataset.

In [ ]:
classes = np.unique(np.concatenate((y_train, y_test), axis=0))

plt.figure()
for c in classes:
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

# Build the model


In [ ]:
module_selection = ("mobilenet_v2", 224, 1280) 
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=False)

print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

EPOCHS = 15

hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

# Train the model

In [ ]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

# Evaluate model on test data

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

# Plot the model's training and validation loss

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()